In [1]:
print("ahmed")

ahmed


In [2]:
from git import Repo

from langchain_classic.text_splitter import Language
from langchain_classic.document_loaders.generic import GenericLoader
from langchain_classic.document_loaders.parsers import LanguageParser
from langchain_classic.vectorstores.faiss import FAISS
from langchain_classic.memory import ConversationSummaryMemory
from langchain_classic.chains import ConversationalRetrievalChain

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq



/home/ahmed/miniconda3/envs/chatapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%pwd

'/home/ahmed/project/GenAI-Repro/research'

## Repo loader

In [ ]:
! mkdir input_repo

In [5]:
repo_path = "input_repo/"

Repo.clone_from(url='https://github.com/Ahmed2797/Network-Security.git',to_path=repo_path)

<git.repo.base.Repo '/home/ahmed/project/GenAI-Repro/research/input_repo/.git'>

## Document loader

In [4]:
path = 'input_repo/Network_Security'

loader = GenericLoader.from_filesystem(path=path,
                                       glob="**/*",
                                       suffixes=['.py'],
                                       parser=LanguageParser(language=Language.PYTHON,
                                                             parser_threshold=100))

In [5]:
documnets = loader.load()

In [6]:
documnets

[Document(metadata={'source': 'input_repo/Network_Security/__init__.py', 'language': <Language.PYTHON: 'python'>}, page_content=''),
 Document(metadata={'source': 'input_repo/Network_Security/data_acess/networkdata_acess.py', 'language': <Language.PYTHON: 'python'>}, page_content='from Network_Security.exception.exception import NetworkSecurityException\nfrom Network_Security.logging.logger import logging\nfrom Network_Security.configeration.mongodb import MongoDBClient \nfrom typing import Optional\nimport pandas as pd\nimport numpy as np \nimport sys\n\n\nclass NetworkData:\n    def __init__(self):\n        try:\n            self.mongo_client = MongoDBClient()   \n        except Exception as e:\n            raise NetworkSecurityException(e, sys)\n\n    def get_dataframe(self, collection_name: str, database_name: Optional[str] = None)->pd.DataFrame:\n        try:\n            if database_name:\n                collection = self.mongo_client.client[database_name][collection_name]\n    

In [7]:
document_spliter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,chunk_size = 500,chunk_overlap = 50
)

In [8]:
document_spliter

In [9]:
text_chunk = document_spliter.split_documents(documnets)

In [10]:
print("text_chunk:",len(text_chunk))

text_chunk: 134


## Embedding Model

In [ ]:
## embeddings=OpenAIEmbeddings(disallowed_special=())

model_name="sentence-transformers/all-MiniLM-l6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)
embedding

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-l6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
vector_store = FAISS.from_documents(text_chunk,embedding)
## vector_store.persist()
vector_store.save_local("faiss_index")

## LLM

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=groq_api_key,
    max_tokens=500
)

In [18]:
memory = ConversationSummaryMemory(llm=llm,memory_key='chat_history',return_messages=True)

In [19]:
retriver = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 50})
search = ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriver,memory=memory)

In [20]:
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7cd77fbaf380>, search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 50})

In [21]:
search

ConversationalRetrievalChain(memory=ConversationSummaryMemory(llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7cd77e9e2de0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7cd7b58025d0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=500), chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'), verbose=False, combine_docs_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additiona

In [22]:
question = 'whats get_feature_extract_data'
result = search(question)
result

/tmp/ipykernel_32548/329157500.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = search(question)


{'question': 'whats get_feature_extract_data',
 'chat_history': [SystemMessage(content='', additional_kwargs={}, response_metadata={})],
 'answer': 'Based on the provided context, `get_feature_extract_data` is a method of an object ( likely a class) that appears to be responsible for extracting data from a MongoDB database and processing it for feature extraction.\n\nHere\'s a high-level overview of what this method seems to be doing:\n\n1. It logs an informational message to indicate that it\'s extracting data from MongoDB.\n2. It creates an instance of `NetworkData` and calls its `get_dataframe` method to retrieve a DataFrame from a MongoDB collection specified in `self.ingestion_config.data_ingestion_collection_path`.\n3. It defines a path for feature data extraction in `self.ingestion_config.data_ingestion_feature_path`.\n4. It doesn\'t seem to be using the feature data path for anything in the provided snippet.\n5. It defines some metrics from an artifact called `metrics_artifact`

In [23]:
result['answer']

'Based on the provided context, `get_feature_extract_data` is a method of an object ( likely a class) that appears to be responsible for extracting data from a MongoDB database and processing it for feature extraction.\n\nHere\'s a high-level overview of what this method seems to be doing:\n\n1. It logs an informational message to indicate that it\'s extracting data from MongoDB.\n2. It creates an instance of `NetworkData` and calls its `get_dataframe` method to retrieve a DataFrame from a MongoDB collection specified in `self.ingestion_config.data_ingestion_collection_path`.\n3. It defines a path for feature data extraction in `self.ingestion_config.data_ingestion_feature_path`.\n4. It doesn\'t seem to be using the feature data path for anything in the provided snippet.\n5. It defines some metrics from an artifact called `metrics_artifact` (not shown in the snippet).\n6. It defines a split ratio `DATA_INGESTION_SPLIT_RATIO`.\n7. It creates a preprocessor object using the `get_data_tra